In [1]:
from typing import List
from pathlib import Path
from argparse import ArgumentParser
import arcpy

print(arcpy.CheckOutExtension("network"))

# set Project database's directory as workspace
arcpy.env.workspace = "TravelProjectPy.gdb"

CheckedOut


In [ ]:
# MANUALLY ADD & MAKE basic data:
# * seoul_roads (planarized) and hot_places
# * make network dataset, set properties (travel mode);

# Build Network Dataset Layer

In [2]:
# configs
nd = "travel/travel_nd" # network dataset
nd_layer_name = "travel_nd" # layer name

# make and build
arcpy.nax.MakeNetworkDatasetLayer(nd, nd_layer_name)
arcpy.nax.BuildNetwork(nd_layer_name)

# get travel mode "walking"
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["walking"]

# Service Area Analysis

In [3]:
# configs
service_area = arcpy.nax.ServiceArea(nd_layer_name) # set nd layer
service_area.defaultImpedanceCutoffs = [100, 500, 1000] # cut offs  
service_area.travelMode = travel_mode # use travel mode "walking"
service_area.distanceUnits = arcpy.nax.DistanceUnits.Meters # distance unit
service_area.outputType = arcpy.nax.ServiceAreaOutputType.Polygons
service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Overlap

# set facilities
input_facilities = "travel/hot_places"
service_area.load(arcpy.nax.ServiceAreaInputDataType.Facilities, input_facilities)

# run
result = service_area.solve()

In [4]:
# export the results to a feature class

output_polygons = "travel/service_area_py"

if result.solveSucceeded:
    result.export(arcpy.nax.ServiceAreaOutputDataType.Polygons, output_polygons)
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

# Spatial Join Analysis

In [5]:
# configs
target_features = output_polygons
join_features = "travel/hot_places"
out_feature_class = "travel/service_area_join_py"

# add "place_id" to the join feature
arcpy.management.AddField(join_features, "place_id", "LONG")

with arcpy.da.UpdateCursor(join_features, ["ObjectID", "place_id"]) as c:
    for row in c:
        row[1] = row[0]
        c.updateRow(row)

# run
arcpy.analysis.SpatialJoin(
    target_features, join_features, out_feature_class,
    join_operation="JOIN_ONE_TO_MANY",
    match_option="CONTAINS",
)

<Result 'D:\\Dropbox\\my_documents\\some_repositories\\ag-tech-materials\\gis\\arcgis-travel\\TravelProjectPy\\TravelProjectPy.gdb\\travel\\service_area_join_py'>

# LookUp

In [6]:
pid_to_name_dict = {
  1: "세종대왕상", 2: "광화문", 3: "종로구청", 4: "우정총국", 5: "종로타워",
  6: "피마길", 7: "광교", 8: "명동성당", 9: "서울광장", 10: "남대문", 11: "한국은행"
}

def pid_to_name(pid: int) -> str:
  return pid_to_name_dict.get(pid)

def cutoff0_to_rng(cutoff0) -> str:
  if cutoff0==0:
    return "0~100"
  elif cutoff0==50:
    return "100~500"
  else:
    return "500~1000"


def travel_from(
  from_id, cutoffs0: List[int]
) -> List[str]:
  
  cutoff0_places = {}

  for cutoff0 in cutoffs0:
    places = []
    with arcpy.da.SearchCursor(out_feature_class, ["FacilityID", "FromBreak", "place_id"]) as c:
      for (fid, from_, pid) in c:
          if fid==from_id and pid!=from_id and from_==cutoff0 and pid is not None:
              places.append(pid)

    cutoff0_places[cutoff0] = places
    
  # print result
  for cutoff0, places in cutoff0_places.items():
    cutoff_ = cutoff0_to_rng(cutoff0)
    if len(places)==0:
      print(f"It's hard to go to somewhere within {cutoff_} from {pid_to_name(from_id)}")
    else:
      print(f"Within {cutoff_}, You can go to... {', '.join([pid_to_name(pid) for pid in places])} from {pid_to_name(from_id)}")

  return cutoff0_places


travel_from(1, [0, 100, 500])
print()
travel_from(6, [0, 100, 500])
print()

It's hard to go to somewhere within 0~100 from 세종대왕상
Within 500~1000, You can go to... 광화문, 종로구청 from 세종대왕상
Within 500~1000, You can go to... 우정총국, 종로타워, 피마길, 광교 from 세종대왕상

It's hard to go to somewhere within 0~100 from 피마길
Within 500~1000, You can go to... 종로구청, 종로타워, 광교 from 피마길
Within 500~1000, You can go to... 세종대왕상, 광화문, 우정총국, 서울광장 from 피마길

